In [92]:
from pyspark.sql import SparkSession
import sys
spark = SparkSession.builder.getOrCreate()
from pymongo import MongoClient
import hdfs
from pywebhdfs.webhdfs import PyWebHdfsClient
from pprint import pprint
import json
from datetime import datetime as dt
import datetime
import pandas as pd
import pytz

In [93]:
hdfs = PyWebHdfsClient(host='hadoop-master',port='9870', user_name='kafka-data')  # your Namenode IP & username here
my_dir = '/user/kafka-data'
data_files = [i["pathSuffix"] for i in hdfs.list_dir(my_dir)["FileStatuses"]["FileStatus"]]
pprint([data_files, f' nombres de fichiers: {len(data_files)}, len(data_files)*50], compact=True)

[['data-0.json', 'data-1.json', 'data-10.json', 'data-11.json', 'data-12.json',
  'data-13.json', 'data-14.json', 'data-15.json', 'data-16.json',
  'data-17.json', 'data-18.json', 'data-19.json', 'data-2.json', 'data-20.json',
  'data-21.json', 'data-22.json', 'data-23.json', 'data-24.json',
  'data-25.json', 'data-26.json', 'data-27.json', 'data-28.json',
  'data-29.json', 'data-3.json', 'data-30.json', 'data-31.json', 'data-32.json',
  'data-33.json', 'data-34.json', 'data-35.json', 'data-36.json',
  'data-37.json', 'data-38.json', 'data-39.json', 'data-4.json', 'data-40.json',
  'data-41.json', 'data-42.json', 'data-43.json', 'data-44.json',
  'data-45.json', 'data-46.json', 'data-47.json', 'data-48.json',
  'data-49.json', 'data-5.json', 'data-50.json', 'data-51.json', 'data-52.json',
  'data-53.json', 'data-54.json', 'data-55.json', 'data-56.json',
  'data-57.json', 'data-58.json', 'data-59.json', 'data-6.json', 'data-60.json',
  'data-61.json', 'data-62.json', 'data-63.json', 'da

In [94]:
conn = MongoClient('mongo', 27017)
db = conn.tweets
collection = db.tweet

In [90]:
collection.drop()

for i in data_files:
    df = spark.read.json(f'hdfs://hadoop-master:9000/user/kafka-data/{i}')
    json_data = df.toJSON().collect()
    for i in json_data:
        data = json.loads(i)
        hashtags = [i for i in data["hashtags"]]
        for h in hashtags:
            hours = 2
            hours_added = datetime.timedelta(hours = hours)
            new_datetime = dt.fromtimestamp(int(data["timestamp"])/1000)
            new_datetime = new_datetime + hours_added

            uniqued_hashtag = {"hashtag": h, "datetime": new_datetime}
            collection.insert_one(uniqued_hashtag)
        
cursor = collection.find()
for record in cursor:
    print(record)

{'_id': ObjectId('60d5b9e8e0a097dedfdae017'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 39, 878000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae018'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 86000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae019'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 374000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae01a'), 'hashtag': 'NazisrausausdenStadien', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae01b'), 'hashtag': 'HUNGER', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae01c'), 'hashtag': 'Goretzka', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae01d'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b9e8e0a097dedfdae01e'), 'hashtag': '

{'_id': ObjectId('60d5b9eae0a097dedfdae625'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 71000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae626'), 'hashtag': 'WhistlePodu', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 71000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae627'), 'hashtag': 'Yellove', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 71000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae628'), 'hashtag': 'kadaikutty', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 71000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae629'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 366000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae62a'), 'hashtag': 'WhistlePodu', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 366000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae62b'), 'hashtag': 'Yellove', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 11, 366000)}
{'_id': ObjectId('60d5b9eae0a097dedfdae62c'), 'hashtag': 'kadaikutt

{'_id': ObjectId('60d5b9ebe0a097dedfdaec45'), 'hashtag': 'Euro2021', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 44, 720000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec46'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 45, 226000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec47'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 45, 979000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec48'), 'hashtag': 'Euro2021', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 45, 979000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec49'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 46, 79000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec4a'), 'hashtag': 'Heineken', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 46, 79000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec4b'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 46, 981000)}
{'_id': ObjectId('60d5b9ebe0a097dedfdaec4c'), 'hashtag': 'POR', 'dateti

{'_id': ObjectId('60d5b9ece0a097dedfdaf290'), 'hashtag': 'por', 'datetime': datetime.datetime(2021, 6, 25, 11, 28, 0, 517000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf291'), 'hashtag': 'ForDanmark', 'datetime': datetime.datetime(2021, 6, 25, 11, 28, 1, 349000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf292'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 28, 1, 349000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf293'), 'hashtag': 'DEN', 'datetime': datetime.datetime(2021, 6, 25, 11, 28, 1, 349000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf294'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 28, 3, 10000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf295'), 'hashtag': 'ItaliaAustria', 'datetime': datetime.datetime(2021, 6, 25, 11, 58, 8, 7000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf296'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 58, 8, 481000)}
{'_id': ObjectId('60d5b9ece0a097dedfdaf297'), 'hashtag': 'MattHancock', 'datetime'

{'_id': ObjectId('60d5b9ede0a097dedfdaf834'), 'hashtag': 'France', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 8000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf835'), 'hashtag': 'TousContreMacron', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 8000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf836'), 'hashtag': 'MacronDestitution', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 8000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf837'), 'hashtag': 'Confinement', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 8000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf838'), 'hashtag': 'Resistance', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 8000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf839'), 'hashtag': 'Euro2020', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 727000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf83a'), 'hashtag': 'ESP', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 2, 727000)}
{'_id': ObjectId('60d5b9ede0a097dedfdaf83b'), 'hashtag': 'podcast', 'da

{'_id': ObjectId('60d5b9eee0a097dedfdafe44'), 'hashtag': 'BigOneExchange', 'datetime': datetime.datetime(2021, 6, 25, 13, 11, 23, 644000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe45'), 'hashtag': 'HarishSalve', 'datetime': datetime.datetime(2021, 6, 25, 13, 11, 23, 714000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe46'), 'hashtag': 'संविदा_नहीं_स्थाई_कंप्यूटर_शिक्षक', 'datetime': datetime.datetime(2021, 6, 25, 13, 11, 23, 706000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe47'), 'hashtag': 'Euro2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 29, 32, 348000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe48'), 'hashtag': 'Euro2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 29, 33, 558000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe49'), 'hashtag': 'bière', 'datetime': datetime.datetime(2021, 6, 25, 11, 29, 33, 558000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe4a'), 'hashtag': 'Pogba', 'datetime': datetime.datetime(2021, 6, 25, 11, 29, 33, 558000)}
{'_id': ObjectId('60d5b9eee0a097dedfdafe4

In [86]:
hours_removed = datetime.timedelta(hours = 1)
today = dt.now(pytz.timezone('Europe/Paris'))
new_datetime = today - hours_removed
new_datetime

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'